## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [3]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [4]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [5]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/TOBS_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [6]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
# print Eig
Eig.head()

,0,1,2,3
0,0.028206,-0.098837,0.120035,0.294161
1,-0.464753,0.064281,-0.010572,0.021124
2,0.038852,0.029542,0.059386,-0.018006
3,0.025581,-0.070210,0.027941,-0.147139
4,0.040537,0.037199,-0.070976,-0.326079


In [7]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00087395', u'USC00087137', u'USC00088782', u'USC00083321', u'USC00083322']


,0,1,2,3
station,,,,
USC00087395,0.028206,-0.098837,0.120035,0.294161
USC00087137,-0.464753,0.064281,-0.010572,0.021124
USC00088782,0.038852,0.029542,0.059386,-0.018006
USC00083321,0.025581,-0.070210,0.027941,-0.147139
USC00083322,0.040537,0.037199,-0.070976,-0.326079


In [8]:
table=Eig.join(Stations,how='left')
# print table

In [9]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00087395,PUNTA GORDA,26.9333,-82.0500,3.0,0.028206,-0.098837,0.120035,0.294161
USC00087137,PINELLAS PARK,27.8333,-82.7000,4.6,-0.464753,0.064281,-0.010572,0.021124
USC00088782,TAMPA BAY AREA WFO,27.7053,-82.4006,12.2,0.038852,0.029542,0.059386,-0.018006
USC00083321,GAINESVILLE 3 WSW,29.6333,-82.3667,29.3,0.025581,-0.070210,0.027941,-0.147139


In [10]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
print colors

[u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']


## Map

In [11]:
min_lat,max_lat,min_long,max_long = box =  (26.3000, 30.400, -81.70, -83.5)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 8
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=1.5 * abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [25]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined